In [ ]:
import scipy as sp
import numpy as np

# Метод градиентного спуска
Метод градиентного спуска позволяет искать локальный минимум функции нескольких переменных. Идея алгоритма заключается в том, чтобы на каждой итерации двигаться от текущей точки в направлении убывания, то есть в направлении, противоположном градиенту функции: $s_{(k)} = -grad(f(x_{(k)}))$.
Однако необходимо найти коэффициент $coef$ с которым мы будем идти по этому направлению: $x_{(k + 1)}  = x_{(k)} + coef * s_{(k)}$.



In [121]:
def gradient_descent(f_runnable, x0, f_step, eps, step_size=0.0):
    """Gradient descent method.

    Keyword arguments:
    f_runnable -- objective function (for which we want to find the minimum)
    x0 -- starting point
    f_step -- method for finding the learning (step) rate
    eps -- calculation accuracy.


    Returns a numpy array containing:
    the x coordinates (at which the minimum is reached),
    the minimum value,
    the number of iterations,
    the number of gradient calculations,
    the number of function calculations.
    """

    x_res = x0
    x_last = x0
    cnt_grad = 0
    cnt_func = 0
    cnt_iter = 0
    exception = ''

    while True:
      cnt_iter += 1

      try:
        gradient = sp.optimize.approx_fprime(x_last, f_runnable, eps)
        if any(np.isnan(gradient[i]) for i in range(len(gradient))):
          exception = 'Overflow'
      except Exception as e:
        exception = str(e)

      if exception != '':
        break
      cnt_grad += 1

      eval_step = f_step(step_size, x_last, f_runnable, gradient, eps)
      step = eval_step[0]
      cnt_func += eval_step[1]
      x_new = [x - step * grad_i for x, grad_i in zip(x_last, gradient)]

      cnt_func += 2
      if abs(f_runnable(x_new) - f_runnable(x_last)) < eps:
        break;

      x_last = x_res
      x_res = x_new

    return np.array([*x_res, f_runnable(x_res), cnt_iter, cnt_grad, cnt_func, exception]);

# Различные методы поиска коээфициента шага на каждой итерации (learning rate)

1. Константный шаг, заданный в диапазоне от 0 до 1. Большие значения соответствуют большему шагу во время одной итерации, соответственно алгоритм будет работать быстрее (так как для поиска минимума потребуется меньше итераций), но ответ будет менее точный. Малые значения коэффициента соответствуют большей точности ответа, но потребуется так же и больше итераций для его поиска.

2. Метод золотого сечения – метод поиска экстремума функции одной переменной, где на каждой итерации мы имеем две точки (при этом на на первой итерации и далее мы считаем только одну, вторая сохранена с прошлой итерации и делит уже новый отрезок в отношении золотого сечения), после из трех промежутков отбрасываем соответствующий левый или правый.
3. Метод дихотомии.

In [89]:
def constant_step(step_size, x_last, f_runnable, gradient, eps):
    """The function returns a constant specified step."""
    return np.array([step_size, 0]);

def gradient_descent_with_constant_step(f_runnable, x0, step_size, eps):
    """Gradient descent method with constant learning rate."""
    return gradient_descent(f_runnable, x0, constant_step, eps, step_size)

def golden_ratio_step(step_size, x_last, f_runnable, gradient, eps):
    """The function returns the step calculated using the golden ratio method."""
    cnt_func = 0
    phi = (1 + 5 ** 0.5) / 2

    a = 0
    b = 1
    ml = b - (b - a) / phi
    mr = a + (b - a) / phi
    xl = [x - ml * grad_i for x, grad_i in zip(x_last, gradient)]
    xr = [x - mr * grad_i for x, grad_i in zip(x_last, gradient)]
    lvalue = f_runnable(xl)
    rvalue = f_runnable(xr)
    cnt_func += 2

    while (b - a > eps):
      if lvalue < rvalue:
        b = mr
        mr = ml
        ml = b - (b - a) / phi
        rvalue = lvalue
        xr = xl
        xl = [x - ml * grad_i for x, grad_i in zip(x_last, gradient)]
        lvalue = f_runnable(xl)
        cnt_func += 1
      else:
        a = ml
        ml = mr
        mr = a + (b - a) / phi
        lvalue = rvalue
        xl = xr
        xr = [x - mr * grad_i for x, grad_i in zip(x_last, gradient)]
        rvalue = f_runnable(xr)
        cnt_func += 1

    return np.array([a, cnt_func]);

def gradient_descent_with_golden_ratio_step(f_runnable, x0, eps):
    """Gradient descent method with learning rate calculated using the golden ratio method."""
    return gradient_descent(f_runnable, x0, golden_ratio_step, eps)

def descent_step(step_size, x_last, f_runnable, gradient, eps):
    """The function returns the step calculated using the descent half method."""
    cnt_func = 0
    a = 0
    b = 1

    while (b - a > 3 * eps):
      ml = (a + b) / 2 - eps
      mr = (a + b) / 2 + eps
      xl = [x - ml * grad_i for x, grad_i in zip(x_last, gradient)]
      xr = [x - mr * grad_i for x, grad_i in zip(x_last, gradient)]
      cnt_func+=2
      if f_runnable(xl) <= f_runnable(xr):
        b = mr
      else:
        a = ml

    return np.array([a, cnt_func]);

def gradient_descent_with_descent_half_step(f_runnable, x0, eps):
    """"Gradient descent method with learning rate calculated using the descent method."""
    return gradient_descent(f_runnable, x0, descent_step, eps)

# Тесты

В тестах для сравнения так же используется библиотечный метод Нелдера-Мида.
Метод Нелдера-Мида берет симплекс (выпуклая оболочка n+1 точки в n-мерном пространстве ) и отражает ‘максимальную” точку (точку, значение функции от которой максимально из всех n+1 рассматриваемых значений функций) и либо переходит к новому набору, где вместо “максимальной” точки новая (если новая “меньше” всех остальных), либо сжимает по прямой отражения (до тех пор пока не найдется “меньшая” точка), иначе стягивается к “минимальной” точке из набора.

In [127]:
def message(name, result, len_x):
  print(name)
  if result[len_x + 4] != '':
    print("Search failed. Exception: ", result[len_x + 4])
    print()
    return
  print("----- Argument values = [", ", ".join(str(result[i]) for i in range(len_x)), "]")
  print("----- Function result = ", result[len_x])
  print("----- Iteration count = ", result[len_x + 1])
  print("----- Gradient evaluation count = ", result[len_x + 2])
  print("----- Function evaluation count = ", result[len_x + 3])
  print()

def test(start_point, func, const_step, eps):
  len_x = len(start_point)

  print("Starting point: " + str(start_point))
  print("Const step: ", const_step)
  print("Required accuracy : ", eps)
  print('------------------------------------------')
  print()

  Nelder_Mead_res = sp.optimize.minimize(func, start_point, method="Nelder-Mead")
  np_arr_Nelder_Mead_res = np.array([*Nelder_Mead_res.x, func(Nelder_Mead_res.x), str(Nelder_Mead_res.nit), 0, Nelder_Mead_res.nfev, ''])
  message('Nelder-Mead:', np_arr_Nelder_Mead_res, len_x)

  constant_gradient_res = gradient_descent_with_constant_step(func, start_point, const_step, eps)
  message('Constant gradient:', constant_gradient_res, len_x)

  golden_ratio_gradient_res = gradient_descent_with_golden_ratio_step(func, start_point, eps)
  message('Golden ratio gradient:', golden_ratio_gradient_res, len_x)

  half_gradient_res = gradient_descent_with_descent_half_step(func, start_point, eps)
  message('Half gradient:', half_gradient_res, len_x)

In [141]:
# Пример тестирования

def test_func(x):
  return x[0] ** 4 + x[1] ** 2 - x[0] * x[1]

test(np.array([50, -50]), test_func, 0.2, 1e-9)

Starting point: [ 50 -50]
Const step:  0.2
Required accuracy :  1e-09
------------------------------------------

Nelder-Mead:
----- Argument values = [ -0.3535149135682817, -0.17678290062083504 ]
----- Function result =  -0.015624998612451008
----- Iteration count =  59
----- Gradient evaluation count =  0
----- Function evaluation count =  112

Constant gradient:
Search failed. Exception:  Overflow

Golden ratio gradient:
----- Argument values = [ -0.35358702968067307, -0.1767935076541603 ]
----- Function result =  -0.01562499943415202
----- Iteration count =  41
----- Gradient evaluation count =  41
----- Function evaluation count =  1968.0

Half gradient:
----- Argument values = [ -0.35358897478003676, -0.17679698751933032 ]
----- Function result =  -0.01562499936056845
----- Iteration count =  41
----- Gradient evaluation count =  41
----- Function evaluation count =  2542.0



<ipython-input-141-db01bf624696>:4: RuntimeWarning: overflow encountered in scalar power
  return x[0] ** 4 + x[1] ** 2 - x[0] * x[1]
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


# Выводы

1. Во-первых, заметим главное преимущество, которое мы уже знали, метод Нелдера-Мида, в отличие от градиентного спуска, не использует производные функции и, следовательно, может использоваться с негладкими и/или зашумленным функциям. Метод может быть менее эффективным в задачах с большим количеством переменных или сложной функцией.
2. Метод градиентного спуска может не сходится. Он может не сходится, например, в случае, когда используется постоянный шаг и его значение большое для конкретной функции. Например, в функции $f(x, y) = x^2 + 100y^2$ при значении $> 0.1$. Это связано с тем, что при таком шаге смещение может перескакивать область с меньшими значениями, не заходить в нее и зациклится\застрять на границе, не найдя минимум.
3. Градиентный спуск с постоянным шагом работает сильно хуже спуска на основе золотого сечения, как минимум на порядок. Это связано с тем, что использование золотого сечения позволяет подстраивать длину перемещения: в отдаленных от минимума окрестностях с быстрым возрастанием функции можно двигаться к минимум большими шагами, но при этом не пропустить этот минимум.
4. Градиентный спуск с использованием золотого сечения вычисляет значение функции минимум на порядок больше раз в сравнении с количеством итераций. Градиентный спуск с постоянным шагом использует ровно $2*countIter$ вычислений функции и, следовательно, при удачно выбранном шаге может быть эффективнее спуска с использованием золотого сечения на сложно (=затратно) вычислимых функциях.
5. Большой минус метода градиентного спуска с постоянным шагом – это необходимость настройки параметра шага, который  может быть сложно подобрать.
6. Метод дихотомии и метод золотого сечения примерно одинаково работают, различие только в том, что в методе дихотомии сравниваются значения $f(middle + eps)$ и $f(middle - eps)$, тем самым рассматриваемый промежуток уменьшается в примерно 2 раза, в то время как в золотом сечении промежуток уменьшается в 1.618.. раз, но необходимо вычислять на каждом следующем шаге только одно значение функции, следовательно, и всего вычислений при использовании метода золотого сечения меньше.
